# SARIMAX forecasting on PV

#### Import data

In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd

import DataRetriever as dr

retriever = dr.DataRetriever()
year2 = retriever.get_data("All-Subsystems-hour-Year2.pkl")
year2

,Timestamp,Load_LatentHeatWaterVolume,Load_RefrigeratorTemp,Load_StatusBA1Lights,Load_StatusKitchenLightsA,Load_StatusKitchenLightsB,Load_StatusKitchenLightsC,Load_StatusDRLights,Load_StatusLRLights3,Load_StatusEntryHallLights,...,SHW_GlycolFlowHXCoriolisSHW,SHW_WaterFlowHXCoriolisSHW,SHW_GlycolFlowRateHXCoriolisSHW,SHW_WaterFlowRateHXCoriolisSHW,HVAC_HeatPumpIndoorUnitPower,HVAC_HeatPumpOutdoorUnitPower,HVAC_DehumidifierPower,HVAC_DehumidifierInletAirTemp,HVAC_DehumidifierExitAirTemp,HVAC_DehumidifierAirflow
Timestamp,,,,,,,,,,,,,,,,,,,,,
2015-02-01 00:00:00,2015-02-01 00:00:00,0.060105,4.957915,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,NaN,NaN,NaN,NaN,175.449170,1045.817746,4.498593,72.228102,74.959034,0.0
2015-02-01 01:00:00,2015-02-01 01:00:00,0.131439,4.904702,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,NaN,NaN,NaN,NaN,391.606267,1042.026917,4.442783,71.963450,73.979483,0.0
2015-02-01 02:00:00,2015-02-01 02:00:00,0.194187,5.054887,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,NaN,NaN,NaN,NaN,388.484683,1023.145217,4.444083,71.921017,73.818717,0.0
2015-02-01 03:00:00,2015-02-01 03:00:00,0.258256,4.991525,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,NaN,NaN,NaN,NaN,175.792833,1066.177350,4.467983,71.844683,74.535050,0.0
2015-02-01 04:00:00,2015-02-01 04:00:00,0.322324,5.040868,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,NaN,NaN,NaN,NaN,371.120983,1081.112783,4.466550,71.959983,74.741467,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-01-31 19:00:00,2016-01-31 19:00:00,1.472255,4.810046,0.0,0.750000,0.750000,0.750000,0.766667,0.750000,0.0,...,419.930636,942.630328,0.001339,0.000336,11.229733,22.489150,4.243500,67.764183,68.643883,0.0
2016-01-31 20:00:00,2016-01-31 20:00:00,1.546231,4.917617,0.0,0.750000,0.750000,0.750000,0.483333,1.000000,0.0,...,419.930636,942.630328,0.001503,0.000385,11.366567,22.483300,4.275667,67.889300,68.772900,0.0
2016-01-31 21:00:00,2016-01-31 21:00:00,1.617565,4.820712,0.0,1.000000,1.000000,1.000000,0.000000,1.000000,0.0,...,419.930636,942.630328,0.001470,0.000437,11.448617,22.509617,4.297500,67.785950,68.651417,0.0


In [2]:
PV_ATTRIBUTES = retriever.get_attributes(file_name='producing_attributes.pkl')

gen_df = retriever.get_data(file_name='All-Subsystems-hour-Year2.pkl')[PV_ATTRIBUTES].sum(axis=1).clip(lower=0) / 1000
gen_df = gen_df.rename("Generated Energy", inplace=True).to_frame()

#### Train, Validation and Test data

In [55]:
date =  pd.Timestamp(year=2016, month=1, day=17, hour=0, minute=0, second=0)
validation_start, validation_end = date + pd.Timedelta(hours=1), date + pd.Timedelta(days=3)
test_start, test_end = date + pd.Timedelta(days=3, hours=1), date + pd.Timedelta(days=6)

train_df = gen_df[:date]
validation_df = gen_df[validation_start: validation_end]
test_df = gen_df[test_start: test_end]

#### ARIMA

In [4]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

def arimamodel(*, training_data, trend='c', p, d, q, P=0, D=0, Q=0, m=0,):

    model = SARIMAX(training_data, trend=trend, order=(p, d, q), seasonal_order=(P, D, Q, m))
    fitted_model = model.fit(maxiter=15)

    return fitted_model

def arima_forecast(data, fitted_model, *, start_index, end_index):
    df = data.copy()
    df['Prediction'] = fitted_model.predict(start = start_index, end = end_index)

    return df

In [5]:
model1 = arimamodel(training_data=train_df['Generated Energy'], trend='c', p=4, d=0, q=0)
model1.summary()

 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            6     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  1.20982D+00    |proj g|=  5.21012D-04

At iterate    5    f=  1.20982D+00    |proj g|=  2.65161D-05

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    6      5      8      1     0     0   2.652D-05   1.210D+00
  F =   1.2098169949547373     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             


<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:       Generated Energy   No. Observations:                 8401
Model:               SARIMAX(4, 0, 0)   Log Likelihood              -10163.673
Date:                Tue, 17 May 2022   AIC                          20339.345
Time:                        10:37:25   BIC                          20381.562
Sample:                    02-01-2015   HQIC                         20353.760
                         - 01-17-2016                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      0.2718      0.019     13.964      0.000       0.234       0.310
ar.L1          1.2654      0.007    173.515      0.000       1.251       1.280
ar.L2         -0.2736      0.009    -31.027      0.000      -0.291      -0.256
ar.L3         -0.0090      0.011     -0.825      0.409      -0.030       0.012
ar.L4         -0.1511      0.008    -18.787      0.000      -0.167      -0.135
sigma2         0.6580      0.006    110.405      0.000       0.646       0.670
===================================================================================
Ljung-Box (L1) (Q):                   0.03   Jarque-Bera (JB):             33497.66
Prob(Q):                              0.86   Prob(JB):                         0.00
Heteroskedasticity (H):               0.70   Skew:                             0.77
Prob(H) (two-sided):                  0.00   Kurtosis:                        12.66
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [6]:
y_pred = model1.predict(start=len(train_df), end=len(train_df) + len(validation_df) - 1)
y_pred

2016-01-17 01:00:00    0.287292
2016-01-17 02:00:00    0.627311
2016-01-17 03:00:00    0.984047
2016-01-17 04:00:00    1.340022
2016-01-17 05:00:00    1.649223
                         ...   
2016-01-19 20:00:00    1.615892
2016-01-19 21:00:00    1.615814
2016-01-19 22:00:00    1.615708
2016-01-19 23:00:00    1.615589
2016-01-20 00:00:00    1.615474
Freq: H, Name: predicted_mean, Length: 72, dtype: float64

In [7]:
from sklearn.metrics import mean_squared_error

model1_rmse = mean_squared_error(validation_df, y_pred, squared=False) #squared=False returns RMSE, True returns MSE
model1_rmse

2.422413103935108

## Seasonal ARIMA

## SARIMA(4, 0, 0, 2, 0, 0, 24) + SARIMA(4, 0, 0, 0, 1, 2, 24)

In [79]:
#model2 = arimamodel(training_data=train_df['Generated Energy'], trend='n', p=4, d=0, q=0, P=2, D=0, Q=0, m=24)
model2_optimised = arimamodel(training_data=train_df['Generated Energy'], trend='n', p=4, d=0, q=0, P=0, D=1, Q=2, m=24)
model2_optimised.summary()

 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            7     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  1.13149D+00    |proj g|=  4.16429D-01

At iterate    5    f=  1.06504D+00    |proj g|=  5.87239D-02

At iterate   10    f=  1.04225D+00    |proj g|=  2.66927D-02


KeyboardInterrupt: 

In [ ]:
y_pred = model2_optimised.predict(start=len(train_df), end=len(train_df) + len(validation_df) - 1)
y_pred

In [ ]:
model2_rmse = mean_squared_error(validation_df, y_pred, squared=False) #squared=False returns RMSE, True returns MSE
model2_rmse

### SARIMAX(4, 0, 0, 0, 1, 2, 24)

In [18]:
from sklearn.preprocessing import OneHotEncoder

gen_df["Hour"] = gen_df.index.hour

ohe = OneHotEncoder(sparse=False)
hot_np = ohe.fit_transform(gen_df[["Hour"]])
hot = pd.DataFrame(data=hot_np, columns=ohe.get_feature_names_out())
hot

,Hour_0,Hour_1,Hour_2,Hour_3,Hour_4,Hour_5,Hour_6,Hour_7,Hour_8,Hour_9,...,Hour_14,Hour_15,Hour_16,Hour_17,Hour_18,Hour_19,Hour_20,Hour_21,Hour_22,Hour_23
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
8756,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
8757,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
8758,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [17]:
gen_df_no_changes = gen_df["Generated Energy"][:test_df.index[-1]].to_frame().reset_index()
gen_df_no_changes

,Timestamp,Generated Energy
0,2015-02-01 00:00:00,0.018496
1,2015-02-01 01:00:00,0.018332
2,2015-02-01 02:00:00,0.018385
3,2015-02-01 03:00:00,0.018502
4,2015-02-01 04:00:00,0.018524
...,...,...
8540,2016-01-22 20:00:00,0.018398
8541,2016-01-22 21:00:00,0.018487
8542,2016-01-22 22:00:00,0.018362
8543,2016-01-22 23:00:00,0.018211


In [42]:
weather_forecast = pd.read_csv("../_05Forecasting/CLEANED_GAI_2015_2016.csv", index_col=0)
weather_forecast.index.rename("Timestamp", inplace=True)
weather_forecast.index = pd.to_datetime(weather_forecast.index)
weather_forecast

,Condition
Timestamp,
2015-01-31 01:00:00,Fair
2015-01-31 02:00:00,Fair
2015-01-31 03:00:00,Fair
2015-01-31 04:00:00,Fair
2015-01-31 05:00:00,Fair
...,...
2016-01-31 20:00:00,Fair
2016-01-31 21:00:00,Fair
2016-01-31 22:00:00,Fair


In [46]:
ohe = OneHotEncoder(sparse=False)
hot = ohe.fit_transform(gen_df["Hour"].to_frame().merge(weather_forecast, left_index=True, right_index=True))
df_ohe = pd.DataFrame(data=hot, columns=ohe.get_feature_names_out())
df_ohe

,Hour_0,Hour_1,Hour_2,Hour_3,Hour_4,Hour_5,Hour_6,Hour_7,Hour_8,Hour_9,...,Hour_19,Hour_20,Hour_21,Hour_22,Hour_23,Condition_Cloudy,Condition_Fair,Condition_Mostly Cloudy,Condition_Partly Cloudy,Condition_Thunder
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
8756,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
8757,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
8758,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0


In [50]:
exog_df = gen_df_no_changes.join(df_ohe)

In [51]:
exog_df.set_index('Timestamp', inplace=True)
exog_df

,Generated Energy,Hour_0,Hour_1,Hour_2,Hour_3,Hour_4,Hour_5,Hour_6,Hour_7,Hour_8,...,Hour_19,Hour_20,Hour_21,Hour_22,Hour_23,Condition_Cloudy,Condition_Fair,Condition_Mostly Cloudy,Condition_Partly Cloudy,Condition_Thunder
Timestamp,,,,,,,,,,,,,,,,,,,,,
2015-02-01 00:00:00,0.018496,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2015-02-01 01:00:00,0.018332,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2015-02-01 02:00:00,0.018385,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2015-02-01 03:00:00,0.018502,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2015-02-01 04:00:00,0.018524,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-01-22 20:00:00,0.018398,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2016-01-22 21:00:00,0.018487,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2016-01-22 22:00:00,0.018362,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0


In [57]:
exog_train = exog_df[:date]
exog_validation = exog_df[validation_start: validation_end] #Last 3 days of observations after test_df indexes have been dropped
exog_test = exog_df[test_start: test_end] #Last 3 days of observations

exog_train

,Generated Energy,Hour_0,Hour_1,Hour_2,Hour_3,Hour_4,Hour_5,Hour_6,Hour_7,Hour_8,...,Hour_19,Hour_20,Hour_21,Hour_22,Hour_23,Condition_Cloudy,Condition_Fair,Condition_Mostly Cloudy,Condition_Partly Cloudy,Condition_Thunder
Timestamp,,,,,,,,,,,,,,,,,,,,,
2015-02-01 00:00:00,0.018496,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2015-02-01 01:00:00,0.018332,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2015-02-01 02:00:00,0.018385,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2015-02-01 03:00:00,0.018502,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2015-02-01 04:00:00,0.018524,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-01-16 20:00:00,0.018212,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2016-01-16 21:00:00,0.018364,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2016-01-16 22:00:00,0.018465,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0


In [58]:
exog_attributes = list(exog_df.columns)[1:]
exog_attributes

['Hour_0',
 'Hour_1',
 'Hour_2',
 'Hour_3',
 'Hour_4',
 'Hour_5',
 'Hour_6',
 'Hour_7',
 'Hour_8',
 'Hour_9',
 'Hour_10',
 'Hour_11',
 'Hour_12',
 'Hour_13',
 'Hour_14',
 'Hour_15',
 'Hour_16',
 'Hour_17',
 'Hour_18',
 'Hour_19',
 'Hour_20',
 'Hour_21',
 'Hour_22',
 'Hour_23',
 'Condition_Cloudy',
 'Condition_Fair',
 'Condition_Mostly Cloudy',
 'Condition_Partly Cloudy',
 'Condition_Thunder']

In [ ]:
model = SARIMAX(exog_train['Generated Energy'], exog=exog_train[exog_attributes], trend='n',
                  order=(4, 0, 0), seasonal_order=(0, 1, 2, 24))
model3 = model.fit()
model3.summary()

In [ ]:
pred = model3.get_prediction(start=validation_start, end=validation_end,
                             exog=exog_df[exog_attributes][validation_start: validation_end])

In [ ]:
sarimax_predictions = pd.DataFrame(pred.predicted_mean)
sarimax_predictions.rename(columns={'predicted_mean': 'Prediction'}, inplace=True)
sarimax_predictions

In [ ]:
sarimax_conf_interval = pred.conf_int()
sarimax_conf_interval

In [ ]:
y_pred = sarimax_predictions['Prediction'][validation_start : validation_end]

model3_rmse = mean_squared_error(validation_df, y_pred, squared=False) #squared=False returns RMSE, True returns MSE
model3_rmse

In [ ]:
sarimax_predictions['Prediction'][validation_df.index[0] : validation_df.index[-1]]

#### Testing on test set

In [ ]:
forecast_test = arima_forecast(data=gen_df, fitted_model=model2_optimised, start_index=0, end_index=gen_df.index[-1])
forecast_test

In [ ]:
y_pred = forecast_test['Prediction'][test_start : test_end]

model2_test_rmse = mean_squared_error(test_df, y_pred, squared=False) #squared=False returns RMSE, True returns MSE
model2_test_rmse